# HDP Overview

## Introduction

The Hierarchical Dirichlet Process mixture model is given by
$$\begin{aligned}
G_0 | \gamma, H &\sim DP(\gamma, H) \\
G_j | \alpha_0, G_0 &\sim DP(\alpha_0, G_0) \\
\theta_{ji} | G_j &\sim G_j \\
x_{ji} | \theta_{ji} &\sim F(\theta_{ji})
\end{aligned} $$

This model is able to non-parametrically cluster each group's data while sharing information both between and within groups.  A Dirichlet process is essentially a discrete distribution with atoms drawn from a (not-necessarily discrete) base measure $H$ and gradually decreasing weights determined by the "stick-breaking process."  In the HDP, each group is a Dirichlet process drawn from another DP $G_0$, so these will contain the same atoms as $G_0$ but with different weights:
$$\begin{aligned}
G_0 &= \sum_{k=1}^{\infty} \beta_k \delta(\phi_k) \\
G_j &= \sum_{k=1}^{\infty} \pi_{jk} \delta(\phi_k) \\
\phi_k | H &\sim H
\end{aligned} $$
Additionally, if we define $\beta, \pi_j$ as the collected weights above, it can be shown that these vectors encode a distribution over $\mathbb{Z}^+$ such that $\beta | \gamma \sim GEM(\gamma)$ and $\pi_j | \alpha_0, \beta \sim DP(\alpha_0, \beta)$.

Successive draws from a DP exhibit clustering behavior, since the probability of taking a certain value is a related to the number of previous draws of that value.  This is shown in the hierarchical sense by the *Chinese restaurant franchise* process.  Imagine a group of Chinese restaurants with a certain number of tables at each restaurant.  Let $\phi_k$ be the global dishes, drawn from $H$; $\psi_{jt}$ be the table-specific dishes, drawn from $G_0$; and $\theta_{ji}$ be the customer-specific dishes, drawn from $G_j$.  Denote $z_{ji}$ as the dish index eaten by customer $ji$; $t_{ji}$ as the table index where customer $ji$ sits; $k_{jt}$ be the dish index served at table $jt$; $n_{jtk}$ be the customer counts; and $m_{jk}$ be the table counts.  Then:

$$\begin{aligned}
\theta_{ji} | \text{other } \theta, \alpha_0, G_0 &\sim
    \sum_{t=1}^{m_{j\cdot}} \frac{n_{jt\cdot}}{i-1+\alpha_0} \delta(\psi_{jt}) +
                            \frac{\alpha_0}{i-1+\alpha_0} G_0 \\
\psi_{jt} | \text{other } \psi, \gamma, H &\sim
    \sum_{k=1}^{K} \frac{m_{\cdot k}}{m_{\cdot \cdot} + \gamma} \delta(\phi_k) +
                            \frac{\gamma}{m_{\cdot \cdot} + \gamma} H
\end{aligned} $$

## *( 5.1 )* Posterior Sampling in the Chinese Restaurant Franchise

Choose some base measure $h(\cdot)$ and a conjugate data-generating distribution $f(\cdot | \theta)$.  Important to compute are $f_k^{-x_{ji}}(x_{ji})$, the mixture component of customer $ij$ under $k$, and $f_k^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt})$, the mixture component of table $jt$ under $k$.  This is done by integrating out $\phi_k$ over the joint density of all such points, for example:

$$\begin{aligned}
f_k^{-x_{ji}}(x_{ji}) &= \frac { \int f(x_{ij} | \phi_k) g(k)d\phi_k } { \int g(k)d\phi_k } \\
g(k) &= h(\phi_k) \prod_{j'i' \neq ji, z_{j'i'} = k} f(x_{j'i'} | \phi_k) 
\end{aligned} $$

The corresponding mixture components for a new customer assignment and new table assignment are denoted $f_{k^*}^{-x_{ji}}(x_{ji})$ and $f_{k^*}^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt})$, which are special cases of their the respective $f_k$ component where no data points have $z_{ij} = k^*$.

Using this, we first compute the likelihood of a given point $x_{ji}$ given the current clustering scheme:
$$
p(x_{ji} | t^{-ji}, t_{ji} = t^*, k) =
    \sum_{k=1}^{K} \frac{m_{\cdot k}}{m_{\cdot \cdot} + \gamma} f_k^{-x_{ji}}(x_{ji}) +
                            \frac{\gamma}{m_{\cdot \cdot} + \gamma} f_{k^*}^{-x_{ji}}(x_{ji})
$$

For efficiency, the Gibbs scheme implemented below only samples the $t$ and $k$ indexes (which can later be reverse-engineered to obtain the actual parameters).  The state space of the $k$ values is technically infinite, and the number of tables/dishes currently associated with the data is undefined.  We keep a running list of active $t$ and $k$ values.  Each update step, each customer is assigned either to one of the existing tables or to a new table, and if a customer is assigned to a new table, a new $k$ corresponding value gets drawn; similarly, each table is assigned a dish, either from the existing dishes or with a new dish.  If a table/dish becomes unrepresented in the current scheme, it gets removed from its respective list.  The update full conditionals are:

$$ \begin{aligned}
p(t_{ji} = t | t^{-ji}, k, ...) &\propto \begin{cases}
    n_{jt\cdot}^{-ji} f_{k_{jt}}^{-x_{ji}}(x_{ji}) & t\text{ used}\\
    \alpha_0 p(x_{ji} | ...) & t\text{ new}
    \end{cases} \\
p(k_{jt} = k | t, k^{-jt}) &\propto \begin{cases}
    m_{\cdot k} f_k^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt}) & k\text{ used}\\
    \gamma f_{k^*}^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt}) & k\text{ new}
    \end{cases} \\
\end{aligned} $$

## *( 5.3 )* Posterior Sampling by Direct Assignment

This approach uses the same mixture components $f_k^{-x_{ji}}(x_{ji})$, but instead of sampling the $t$ then $k$ indexes, we directly sample $z_ji$ (the $k$ value for customer $ji$) and the count value $m_{jk}$ (the number of tables at restaurant $j$ serving $k$), and the weights $\beta$ for the $\phi_k$ atoms in the stick-breaking representation of $G_0$ given below:

$$ \begin{aligned}
\mathbf{\beta} &= (\beta_1, ..., \beta_K, \beta_u) \sim Dir(m_{\cdot 1}, ... m_{\cdot K}, \gamma) \\
G_u &\sim DP(\gamma, H) \\
p(\phi_k | t, k) &\sim h(\phi_k) \prod_{ji : k_{jt_{ji}} = k} f(x_{ji}| \phi_k) \\
G_0 &= \sum_{k=1}^K \beta_k \delta(\phi_k) + \beta_u G_u
\end{aligned} $$

The direct sampling simplifies bookkeeping slightly, as there is no need to keep an explicit map between table $jt$ and its corresponding dish $k$ (this is implied by the count $m_{jk}$ and the weight $\beta_k$).  Additionally, it greatly cuts down on the computation cost.  We still must loop through each customer sequentially and compute the customer-specific mixture components, but there is no longer a need to compute the table-specific components $f_k^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt})$ when assigning the $k$ values.  The update full conditionals are:

$$ \begin{aligned}
p(z_{ji} = k | z^{-ji}, m, \beta) &= \begin{cases}
    (n_{j \cdot k}^{-ji} + \alpha_0 \beta_k) f_k^{x_{ji}}(x_{ji}) & k\text{ used}\\
    \alpha_0 \beta_u f_{k^*}^{-x_{ji}}(x_{ji}) & k\text{ new}
    \end{cases} \\
p(m_{jk} = m | z, m^{-jk}, \beta) &=
    \frac{\Gamma(\alpha_0 \beta_k)}{\Gamma(\alpha_0 \beta_k + n_{j \cdot k})}
    s(n_{j \cdot k}, m) (\alpha_0 \beta_k)^m \\
\mathbf{\beta} | m &\sim Dir(m_{\cdot 1}, ... m_{\cdot K}, \gamma)
\end{aligned} $$

If a dish $k$ becomes unused, its corresponding $\beta_k$ will get temporarily set to 0.  But if that same value reappears, we must redistribute the weights by setting $\beta_k' = b\beta_u$ and $\beta_u' = (1-b)\beta_u$, where $b \sim Beta(1, \gamma)$.

***Note on computing Stirling numbers***

The above function $s(n, m)$ represents an unsigned Stirling number of the first kind.  It has a recursive definition $s(n, m) = s(n-1, m-1) + (n-1) \cdot s(n-1, m)$ with base cases $s(0,0) = s(1,1) = 1$ and $s(n,m) = 0$ when $m = 0$ or $m > n$.  This function grows at a rate similar to factorials, so for numerical stability, it makes sense to be able to compute the natural logarithm.  Define $s_1 = s(n-1, m-1)$ and $s_2 = s(n-1, m)$, and note:

$$ \begin{aligned}
s(n,m) &= s_1 + (n-1) \cdot s_2 \\
\implies \log{s(n,m)} &= \log[s_1 + (n-1) \cdot s_2] \\
    &= \log[e^{\log s_1} + (n-1) \cdot e^{\log s_2}] \\
    &= \log[e^{\log s_1} [1 + (n-1) \cdot e^{\log s_2 - \log s_1}]] \\
    &= \log s_1 + \log[1 + (n-1) \cdot e^{\log s_2 - \log s_1}]
\end{aligned} $$

This reduces the calculation to a function of log-Stirling numbers and the `numpy` routine `log1p`.  The largest number we will need to compute and store is $s_2 / s_1$, which will be much smaller than either $s_1$ or $s_2$.  If the exponential above does still overflow, we can approximate $\log(1+x) \approx \log(x)$ and further reduce the calculation to

$$ \begin{aligned}
\log{s(n,m)} &\approx \log s_1 + \log[(n-1) \cdot e^{\log s_2 - \log s_1}] \\
    &= \log s_1 + \log(n-1) + (\log s_2 - \log s_1) \\
    &= \log(n-1) + \log s_2
\end{aligned} $$

## Distribution-Specific Mixture Components

The only part of this sampling algorithm that depends on the choice of the measures $H$ and $F$ are the mixture components $f_k$, so this is the only part that needs rewritten for each type of model.  Let
$$ \begin{aligned}
V_{kji} &= \{ j'i' : j'i' \neq ji, z_{j'i'} = k \} \\
W_{kjt} &= \{ j'i' : j't_{j'i'} \neq jt, k_{j't_{j'i'}} = k \} \\
T_{jt} &= \{ j'i': t_{j'i'} = jt \} \\
\end{aligned} $$
$V$ is the set of all customers (excluding customer $ij$) eating dish $k$; $W$ is the set of all customers at tables (excluding table $jt$) eating $k$; these correspond to the product terms in the mixture components.  By conjugacy rules and kernel tricks, each $f_k$ can be expressed as functions of these sets.  Each $f_{k^*}$ can be found by using the corresponding $f_k$ formula where $V$ or $W$ is the empty set.

***F = Poisson, H = Gamma***

$$ \begin{aligned}
f(x | \phi_k) &\sim Poisson(\phi_k) \\
h(\phi_k) &\sim Gamma(\alpha, \beta) \\
\\
f_k^{-x_{ji}}(x_{ji}) &= \frac{1}{x_{ji}!} \cdot
    \frac{\Gamma(x_{ji} + \alpha_v)}{(1 + \beta_v)^{x_{ji} + \alpha_v}} \cdot
    \frac{(\beta_v)^{\alpha_v}}{\Gamma(\alpha_v)} \\
f_k^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt}) &= \frac{1}{\prod_T x_t!} \cdot
    \frac{\Gamma(\sum_T x_t + \alpha_w)}{(|T| + \beta_w)^{\sum_T x_t + \alpha_w}} \cdot
    \frac{(\beta_w)^{\alpha_w}}{\Gamma(\alpha_w)} \\
\alpha_v &= \sum_V x_v + \alpha \quad , \quad \beta_v = |V| + \beta \\
\alpha_w &= \sum_W x_w + \alpha \quad , \quad \beta_w = |W| + \beta \\
\end{aligned} $$

***F = Multinomial, H = Dirichlet***

Let $\mathbf{x}$ be a feature vector of length $L$.  The Multinomial/Dirichlet model is given by
$$ \begin{aligned}
f(\mathbf{x} | n, \mathbf{\phi}_k) &\sim Multinomial(n, \mathbf{\phi}_k) \\
h(\mathbf{\phi}_k) &\sim Dirichlet(L, \mathbf{\alpha}) \\
\end{aligned} $$
Note that each $\mathbf{x}_{ji}$ can have a different value of $n_{ji}$, representing different size draws from the multinomial distribution.  But $\phi_k$, representing the relative concentrations of the $L$ components, will be the same.

$$ \begin{aligned}
f_k^{-\mathbf{x}_{ji}}(\mathbf{x}_{ji}) &= \frac{n_{ji}!}{\prod_{\ell=1}^L (\mathbf{x}_{ji})_\ell!} \cdot
    \frac{ \prod \Gamma(\mathbf{\alpha}_{\ell}^{top}) }{ \Gamma(\sum \mathbf{\alpha}_{\ell}^{top}) } \cdot
    \frac{ \Gamma(\sum \mathbf{\alpha}_{\ell}^{bottom}) }{ \prod \Gamma(\mathbf{\alpha}_{\ell}^{bottom}) } \\
\mathbf{\alpha}_{\ell}^{bottom} &= \sum_V (\mathbf{x}_v)_{\ell} + \mathbf{\alpha}_{\ell} \\
\mathbf{\alpha}_{\ell}^{top} &= (\mathbf{x}_{ji})_{\ell} + \mathbf{\alpha}_{\ell}^{bottom} \\
\\
f_k^{-\mathbf{X}_{jt}}(\mathbf{X}_{jt}) &=
    \frac{ \prod_T n_t! }{ \prod_T \prod_{\ell=1}^L (\mathbf{x}_t)_\ell! } \cdot
    \frac{ \prod \Gamma(\mathbf{\alpha}_{\ell}^{top}) }{ \Gamma(\sum \mathbf{\alpha}_{\ell}^{top}) } \cdot
    \frac{ \Gamma(\sum \mathbf{\alpha}_{\ell}^{bottom}) }{ \prod \Gamma(\mathbf{\alpha}_{\ell}^{bottom}) } \\
\mathbf{\alpha}_{\ell}^{bottom} &= \sum_W (\mathbf{x}_w)_{\ell} + \mathbf{\alpha}_{\ell} \\
\mathbf{\alpha}_{\ell}^{top} &= \sum_T (\mathbf{x}_t)_\ell + \mathbf{\alpha}_{\ell}^{bottom} \\
\end{aligned} $$

In the application of latent topic modeling for NLP, restaurant $j$ represents document $j$, and customer $ji$ represents some subset of the word counts in document $j$, where $L$ is the size of the entire vocabulary for the corpus of documents.  If a customer is one word, $\mathbf{x}_{ji}$ is a draw from a categorical distribution of size $L$, a special case of the multinomial; but a customer can also represent the set of all instances of a unique word, the set of words in a sentence, the set of words in a paragraph, etc.

The dishes $k$ represent topics (where $\mathbf{\phi}_k$ is a distribution over the vocabulary), and the tables $t$ represent clusters of words, sentences, paragraphs (however a "customer" is encoded) from a single document.  Due to the nature of the HDP, these topics are shared among documents and among clusters within a document.  If the customer subsets are mutually exclusive, then each section of a document will be assigned to exactly one topic; but if the customers overlap, the same section of a document could be assigned to multiple topics.

***F = Categorical, H = Dirichlet***

This is a special case of the above, where each vector $\mathbf{x} \sim Categorical(L, \mathbf{\phi}_k)$ (i.e., every entry is a zero except for a one in one position).  If we model $f$ in this way, the mixture components can be computed very efficiently.  Let $\mathbf{x}_{ji}$ have a one in position $\ell'$, and note that in this case

$$ \begin{aligned}
f_k^{-\mathbf{x}_{ji}}(\mathbf{x}_{ji}) &= \frac{1!}{1! \prod_{\ell \neq \ell'} 0!} \cdot
    \left[ \frac{ \Gamma(\sum \mathbf{\alpha}_{\ell}^{top}) }
                { \Gamma(\sum \mathbf{\alpha}_{\ell}^{bottom}) } \right]^{-1}
    \prod \frac{ \Gamma(\mathbf{\alpha}_{\ell}^{top}) }{ \Gamma(\mathbf{\alpha}_{\ell}^{bottom}) } \\
    &= \left[ \frac{ \Gamma(1 + |V| + \sum \alpha_{\ell}) }
                   { \Gamma(|V| + \sum \alpha_{\ell}) } \right]^{-1}
    \prod \frac{ \Gamma((\mathbf{x}_{ji})_{\ell} + \mathbf{\alpha}_{\ell}^{bottom}) }
               { \Gamma(\mathbf{\alpha}_{\ell}^{bottom}) } \\
    &= \left[ \frac{ \Gamma(1 + |V| + \sum \alpha_{\ell}) }
                   { \Gamma(|V| + \sum \alpha_{\ell}) } \right]^{-1}
       \frac{ \Gamma(1 + \mathbf{\alpha}_{\ell'}^{bottom}) }
            { \Gamma(\mathbf{\alpha}_{\ell'}^{bottom}) } \\
    &= [|V| + \sum \alpha_{\ell}]^{-1} \cdot \alpha_{\ell'}^{bottom} \\
    &= \frac{|\{v \in V : (x_v)_{\ell'} = 1\}| + \alpha_{\ell'}}
            {|V| + \sum \alpha_{\ell}}
\end{aligned} $$

Similarly, the table-specific mixture component is

$$ \begin{aligned}
f_k^{-\mathbf{X}_{jt}}(\mathbf{X}_{jt})
    &= \left[ \frac{ \Gamma(|T| + |V| + \sum \alpha_{\ell}) }
                   { \Gamma(|V| + \sum \alpha_{\ell}) } \right]^{-1}
       \prod_{\ell' \in \mathcal{L}}
       \frac{ \Gamma( \sum_T (\mathbf{x}_t)_{\ell'} + \sum_W (\mathbf{x}_w)_{\ell'} + \mathbf{\alpha}_{\ell'} ) }
            { \Gamma( \sum_W (\mathbf{x}_w)_{\ell'} + \mathbf{\alpha}_{\ell'} ) } \\
\end{aligned} $$

where $\mathcal{L} = \{\ell' : (x_t)_{\ell'} = 1 \text{ for some } t \in T \}$.  This does not simplify as nicely and remains somewhat computationally intensive, so the direct sampler has a major advantage with this type of model.